In [219]:
# Loading Libraries
import pandas as pd
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import log_hyperu as hyperu
import tgr as tgr

import time

In [ ]:
# Generating the data sets

torch.manual_seed(12345)

# Structure of the data sets
## Set X_A: Sparse Coefficients with many data points
## Set X_B: Dense Coefficients with many data points
## Set X_C: Sparse Coefficients with few data points
## Set X_D: Dense Coefficients with few data points


### Set X_A
variables = 10
sample = 100
true_coefs = torch.tensor([[0],[0],[5.3],[4.2],[0],[0],[6.9],[0],[0],[0]])
X = torch.randn(sample, variables)
Y = X @ true_coefs + torch.randn(sample, 1) * 0.1
print(X)

In [218]:
iterations = 1500
starttime = time.time()
trained_model2, coefs, loss_of_optimization = tgr.TripleGammaModel(X, Y, 1, 0.51, 0.001, 2, True, num_epochs=iterations) # Covariates, Targets, Penalty, a, c, kappa, normalization=True
endtime = time.time()
print(f'With {iterations} iterations, TG Regularization took {round(endtime-starttime,4)} seconds. On this system, that is roughly {(endtime-starttime)/iterations} seconds per iteration.')
coefficients = trained_model2.linear.weight.detach().numpy()
#intercept = trained_model2.linear.bias.item()

# For LASSO Imitation: (X, Y, 1, 1, 30, 1, True, num_epochs=2000) # Covariates, Targets, Penalty, a, c, kappa, normalization=True

#print("Coefficients General Function:", coefficients)
#print("Intercept General Function:", intercept)
print(coefficients[0].tolist())
true_coefs_TGR = coefficients[0]

Epoch [100/1500], Loss: 111.72769165039062


KeyboardInterrupt: 

In [133]:
#LASSO
# Define your model
starttime = time.time()
class LinearRegressionLasso(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegressionLasso, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# Define hyperparameters
input_size = 10
output_size = 1
learning_rate = 0.01
num_epochs = 1000
lambda_lasso = 1 # L1 regularization parameter

# Create the model
model = LinearRegressionLasso(input_size, output_size)

# Define loss function (MSE loss with L1 regularization)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, Y)

    # Total loss with L1 regularization
    l1_norm = sum(torch.linalg.norm(p, 1) for p in model.parameters())

    loss = loss + lambda_lasso * l1_norm
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

endtime = time.time()
print(f'With {num_epochs} iterations, Regular LASSO Regularization took {round(endtime-starttime,4)} seconds. On this system, that is roughly {(endtime-starttime)/num_epochs} seconds per iteration.')

# After training, you can access the learned coefficients
for name, param in model.named_parameters():
    if 'weight' in name:
        print(f'{name}: {param.data.tolist()}')
        coefs_LASSO = param.data.tolist()


Epoch [100/1000], Loss: 10.6036
Epoch [200/1000], Loss: 7.0338
Epoch [300/1000], Loss: 6.6343
Epoch [400/1000], Loss: 6.5833
Epoch [500/1000], Loss: 6.5743
Epoch [600/1000], Loss: 6.5747
Epoch [700/1000], Loss: 6.5778
Epoch [800/1000], Loss: 6.5804
Epoch [900/1000], Loss: 6.5821
Epoch [1000/1000], Loss: 6.5785
With 1000 iterations, Regular LASSO Regularization took 1.3717 seconds. On this system, that is roughly 0.0013716657161712646 seconds per iteration.
linear.weight: [[-0.08131898939609528, 0.11880242824554443, 5.130025863647461, 4.099651336669922, 0.0024799692910164595, -0.008265404962003231, 6.229494094848633, -0.03251353278756142, 0.006142516154795885, -0.11067549884319305]]


In [148]:
def CopmuteRMSE(Y, X, coefficients, method):
    # Variable Type Check
    if not all(isinstance(t, torch.Tensor) for t in [Y, X, coefficients]):
        raise ValueError("Not all inputs are PyTorch tensors!")
    
    Y_hat = X @ coefficients
    Y = torch.squeeze(Y)
    mse = torch.sum((Y - Y_hat)**2)
    rmse = torch.sqrt(mse)
    print(method)
    #print(Y_hat)
    #print(Y)
    print(rmse)

In [182]:
from sklearn import linear_model
ols = linear_model.LinearRegression()
model = ols.fit(X, Y)
ols = model.coef_.tolist()[0] 

In [186]:
print("True COEFS", torch.squeeze(true_coefs).T.tolist())
print("TGR:",torch.tensor(true_coefs_TGR).tolist())
print("LASSO:",torch.squeeze(torch.tensor(coefs_LASSO)).tolist())
print("OLS:", ols)
print("\n")
CopmuteRMSE(Y,X,torch.tensor(true_coefs_TGR), "TGR")
CopmuteRMSE(Y,X,torch.squeeze(torch.tensor(coefs_LASSO)), "LASSO")
CopmuteRMSE(Y,X,torch.squeeze(torch.tensor(ols)), "OLS")

True COEFS [0.0, 0.0, 5.300000190734863, 4.199999809265137, 0.0, 0.0, 6.900000095367432, 0.0, 0.0, 0.0]
TGR: [0.005852716509252787, -0.003165281843394041, 5.298001289367676, 4.181629180908203, -0.006983057130128145, 2.2076303139328957e-05, 6.898454666137695, -0.008982997387647629, -0.00298516359180212, -0.00657022837549448]
LASSO: [-0.08131898939609528, 0.11880242824554443, 5.130025863647461, 4.099651336669922, 0.0024799692910164595, -0.008265404962003231, 6.229494094848633, -0.03251353278756142, 0.006142516154795885, -0.11067549884319305]
OLS: [-0.011616242118179798, 0.005331128835678101, 5.29803991317749, 4.181795120239258, -0.013096123933792114, 0.003976382315158844, 6.89629602432251, -0.014410754665732384, -0.0016365605406463146, -0.008706651628017426]


TGR
tensor(0.9780)
LASSO
tensor(5.8565)
OLS
tensor(0.9644)


In [249]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import tgr as tgr
import log_hyperu as hyperu

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Step 1: Generate synthetic data
n_samples = 100
n_features = 10

# True coefficients with sparsity (many coefficients are zero)
true_coefficients = torch.zeros(n_features)
true_coefficients[:3] = torch.randn(3)

# Generate features
X = torch.randn(n_samples, n_features)

# Generate targets with noise
noise = torch.randn(n_samples) * 0.5
y = X @ true_coefficients + noise

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Implement OLS and Lasso regression using PyTorch

class LinearRegression(nn.Module):
    def __init__(self, n_features):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(n_features, 1, bias=False)
        
    def forward(self, x):
        return self.linear(x)

def train_model(model, X_train, y_train, lr=0.01, n_epochs=1000):
    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    for epoch in range(n_epochs):
        model.train()
        
        optimizer.zero_grad()
        outputs = model(X_train).squeeze()
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
    return model

# Train OLS model
ols_model = LinearRegression(n_features)
ols_model = train_model(ols_model, X_train, y_train)

# Train Lasso model
lasso_model = LinearRegression(n_features)
lasso_reg_strength = 0.5  # Regularization strength

def lasso_loss(output, target, model, lasso_reg_strength):
    mse_loss = nn.MSELoss()(output, target)
    lasso_loss = lasso_reg_strength * torch.norm(model.linear.weight, 1)
    return mse_loss + lasso_loss

optimizer = torch.optim.SGD(lasso_model.parameters(), lr=0.01)
n_epochs = 1000

for epoch in range(n_epochs):
    lasso_model.train()
    
    optimizer.zero_grad()
    outputs = lasso_model(X_train).squeeze()
    loss = lasso_loss(outputs, y_train, lasso_model, lasso_reg_strength)
    loss.backward()
    optimizer.step()


# Train TGR Model
tgr_model = LinearRegression(n_features)
tgr_reg_strength = 0.1

def tgr_loss(output, target, model, lasso_reg_strength):
    mse_loss = nn.MSELoss()(output, target)
    c = 0.001
    a = 0.51
    kappa = 2
    phi = torch.tensor((2*c)/((kappa**2)*a))
    tgr_loss = tgr_reg_strength * torch.sum(-hyperu.log_hyperu(torch.tensor([[c+0.5]]),torch.tensor([[1.5-a]]),(model.linear.weight**2)/(2*phi))+hyperu.log_hyperu(torch.tensor([[c+0.5]]),torch.tensor([[1.5-a]]),torch.tensor([[0.0]])))

    return mse_loss + tgr_loss


optimizer = torch.optim.SGD(tgr_model.parameters(), lr=0.01)
n_epochs = 1500

for epoch in range(n_epochs):
    tgr_model.train()
    
    optimizer.zero_grad()
    outputs = tgr_model(X_train).squeeze()
    loss = tgr_loss(outputs, y_train, tgr_model, lasso_reg_strength)
    loss.backward()
    nn.utils.clip_grad_norm_(tgr_model.parameters(), 1.0)
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
        
# Step 4: Compare performance based on RMSE

def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        predictions = model(X_test).squeeze()
    rmse = torch.sqrt(nn.MSELoss()(predictions, y_test))
    return rmse.item()

ols_rmse = evaluate_model(ols_model, X_test, y_test)
lasso_rmse = evaluate_model(lasso_model, X_test, y_test)
tgr_rmse = evaluate_model(tgr_model, X_test, y_test)

print(f"OLS RMSE: {ols_rmse}")
print(f"Lasso RMSE: {lasso_rmse}")
print(f"TGR RMSE: {tgr_rmse}")

# Step 5: Plot the true and estimated coefficients

# Get the estimated coefficients
ols_coefficients = ols_model.linear.weight.detach().numpy().flatten()
lasso_coefficients = lasso_model.linear.weight.detach().numpy().flatten()
tgr_coefficients = tgr_model.linear.weight.detach().numpy().flatten()
true_coefficients_np = true_coefficients.numpy()

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(true_coefficients_np, label='True Coefficients', marker='o')
plt.plot(ols_coefficients, label='OLS Estimated Coefficients', marker='x')
plt.plot(lasso_coefficients, label='Lasso Estimated Coefficients', marker='*')
plt.plot(tgr_coefficients, label='TGR Estimated Coefficients', marker='.')
plt.xlabel('Feature Index')
plt.ylabel('Coefficient Value')
plt.title('True vs Estimated Coefficients')
plt.legend()
plt.grid(True)
plt.show()


Epoch [100/1000], Loss: 5.327718734741211
Epoch [200/1000], Loss: 5.111759185791016
Epoch [300/1000], Loss: 4.500098705291748
